In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

2025-06-04 18:09:53.095 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 3221.85it/s]


In [2]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from pathlib import Path

settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
    }
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5}

In [5]:
from mltrainer import Trainer
from torch import optim

from RNN import RecurrentNeuralNetworkWithGRU, ModelConfig

config = ModelConfig(
    input_size=3, # vast
    hidden_size=128,
    num_layers=2,
    output_size=20, # vast
    dropout=0.2,
)

model = RecurrentNeuralNetworkWithGRU(
    config=config,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
)

trainer.loop()


2025-06-04 18:29:44.298 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250604-182944
2025-06-04 18:29:44.299 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:02<00:00, 27.96it/s]
2025-06-04 18:29:47.372 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5213 test 2.1004 metric ['0.2234']
100%|██████████| 81/81 [00:02<00:00, 28.89it/s]
2025-06-04 18:29:50.343 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.0382 test 1.8671 metric ['0.3203']
100%|██████████| 81/81 [00:02<00:00, 28.19it/s]
2025-06-04 18:29:53.399 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.5591 test 1.1898 metric ['0.5453']
100%|██████████| 81/81 [00:02<00:00, 29.68it/s]
2025-06-04 18:29:56.304 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.0022 test 0.6874 metric ['0.7750']
100%|██████████| 81/81 [00:02<00:00, 29.96it/s]
2025-06-04 18:29:59.16